In [1]:
import pandas as pd
import numpy as np
import os
from time import time
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, Conv2D, MaxPooling2D
from keras import regularizers, optimizers
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras import backend as K

%matplotlib inline
print(tf.keras.__version__)
print(K.tensorflow_backend._get_available_gpus())

Using TensorFlow backend.


2.1.6-tf
['/job:localhost/replica:0/task:0/device:GPU:0']


In [2]:
# Input constants
HEIGHT = 32
WIDTH = 32
CHANNELS = 3
NUM_CLASSES = 10
NUM_TRAIN_IMAGES = 37500

base_model = ResNet50(weights='imagenet', 
                      include_top=False, 
                      input_shape=(HEIGHT, WIDTH, CHANNELS))

C:\Users\Laurenz\Anaconda3\envs\tfgpu\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [3]:
def append_ext(fn):
    return fn+".png"

traindf=pd.read_csv("./trainLabels.csv",dtype=str)
testdf=pd.read_csv("./sampleSubmission.csv",dtype=str)
traindf["id"]=traindf["id"].apply(append_ext)
testdf["id"]=testdf["id"].apply(append_ext)
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)

In [4]:
BATCH_SIZE = 32

train_generator=datagen.flow_from_dataframe(
preprocessing_function=preprocess_input,
dataframe=traindf,
directory="./train/",
x_col="id",
y_col="label",
subset="training",
batch_size=BATCH_SIZE,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(32,32))

valid_generator=datagen.flow_from_dataframe(
preprocessing_function=preprocess_input,
dataframe=traindf,
directory="./train/",
x_col="id",
y_col="label",
subset="validation",
batch_size=BATCH_SIZE,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(32,32))
test_datagen=ImageDataGenerator(rescale=1./255.)

test_generator=test_datagen.flow_from_dataframe(
preprocessing_function=preprocess_input,
dataframe=testdf,
directory="./test/",
x_col="id",
y_col=None,
batch_size=BATCH_SIZE,
seed=42,
shuffle=False,
class_mode=None,
target_size=(32,32))

Found 37500 images belonging to 10 classes.
Found 12500 images belonging to 10 classes.
Found 300000 images.


// Modell Code für die Version ohne transfer learning

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])

In [6]:
from keras.models import Model

def build_finetune_model(base_model, dropout, fc_layers, num_classes):
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    for fc in fc_layers:
        # New FC layer, random init
        x = Dense(fc, activation='relu')(x) 
        x = Dropout(dropout)(x)

    # New softmax layer
    predictions = Dense(num_classes, activation='sigmoid')(x) 
    
    finetune_model = Model(inputs=base_model.input, outputs=predictions)

    return finetune_model

# Top Layer constants
FC_LAYERS = [64, 64]
DROPOUT = 0.1

finetune_model = build_finetune_model(base_model, 
                                      dropout=DROPOUT, 
                                      fc_layers=FC_LAYERS, 
                                      num_classes=NUM_CLASSES)

In [11]:
# For old model without transfer learning
#
#STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
#STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
#STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
#model.fit_generator(generator=train_generator,
#                    steps_per_epoch=STEP_SIZE_TRAIN,
#                    validation_data=valid_generator,
#                    validation_steps=STEP_SIZE_VALID,
#                    epochs=10
#)

from keras.optimizers import Adam

# Training constants
NUM_EPOCHS = 10
LEARNING_RATE = 0.003
BATCH_SIZE = 32 #todo remove

FILEPATH_SAVE='checkpoints/ResNet50_{}_model_weights.h5'.format(time())
FILEPATH_TENSORBOARD_LOG = 'logdir/{}'.format(int(time()))

checkpoint = ModelCheckpoint(FILEPATH_SAVE,
                             monitor=["acc"],
                             verbose=1,
                             mode='max'
                            )
                                               
early_stopping = EarlyStopping(monitor='acc',
                               min_delta=0.002,
                               patience=4,
                               verbose=0,
                               mode='auto',
                               restore_best_weights=True)
                                               
tensorboard = TensorBoard(log_dir = FILEPATH_TENSORBOARD_LOG, write_images=True)

callbacks_list = [tensorboard]

adam = Adam(lr=LEARNING_RATE)

finetune_model.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6),
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])

history = finetune_model.fit_generator(train_generator, 
                                       epochs=NUM_EPOCHS,
                                       workers=8, 
                                       steps_per_epoch=NUM_TRAIN_IMAGES // BATCH_SIZE, 
                                       shuffle=True,
                                       callbacks=callbacks_list
                                      )

#starting Tensorboard:
#tensorboard --logdir=data/ --host localhost --port 8088
#Navigated the browser to http://localhost:8088

Epoch 1/10
1171/1171 [==============================] - 146s 125ms/step - loss: 1.2637 - acc: 0.5641
Epoch 2/10
1171/1171 [==============================] - 183s 156ms/step - loss: 1.3885 - acc: 0.5343
Epoch 3/10
1171/1171 [==============================] - 144s 123ms/step - loss: 1.3894 - acc: 0.5313
Epoch 4/10
 925/1171 [======================>.......] - ETA: 30s - loss: 1.3673 - acc: 0.5345

KeyboardInterrupt: 

In [ ]:
finetune_model.evaluate_generator(generator=valid_generator,
steps=STEP_SIZE_TEST)

In [ ]:
test_generator.reset()
pred=finetune_model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

In [ ]:
predicted_class_indices=np.argmax(pred,axis=1)

In [ ]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [ ]:
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("results_transfer_learning.csv",index=False)

In [ ]:
finetune_model.save('cifar10_trained_with_transfer_learning.h5')